# Compilation of Quantum Circuits

In classical computing, high-level languages like C++ or Python allow developers to write readable code, which must then be _compiled_ into machine code for execution. This process enables complex software to run efficiently across different platforms.  

Similarly, quantum circuits—sequences of quantum gates operating on qubits—must be compiled for execution on quantum hardware. Quantum processors support only a set of native gates, often with limited qubit connectivity. Therefore, high-level quantum circuits must be translated into a device-compatible form, respecting connectivity constraints while minimizing additional gates. Efficient compilation is critical, as unnecessary gates increase execution time, error rates, and reduce accuracy.

A key step in quantum compilation is _quantum circuit mapping_: assigning logical qubits to physical qubits on the device and enable multi-qubit operations. Since device connectivity is limited in some hardware platforms, the mapping often needs dynamic adjustment using _SWAP_ gates (exchanging the state of neighbouring qubits). Minimizing these additional gates is essential, as even small overheads impact computation accuracy, and the problem is NP-complete.

The _MQT_ provides QMAP for quantum circuit mapping, producing device-compliant circuits with minimal gate overhead. For neutral atom quantum computers, (instead of SWAP optimization) QMAP offers efficient planning of atom movements.

_MQT_ offers many more methods for various compilation tasks, such as Clifford circuit synthesis, determining optimal sub-architectures, compiler optimization, and compilation techniques for different architectures or multi-level quantum systems.

In [ ]:
from qiskit.visualization import plot_gate_map
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.transpiler.coupling import CouplingMap

# Create a 5-qubit device with linear connectivity (0-1-2-3-4)
coupling_map = CouplingMap.from_line(5)
basis_gates = ["u3", "cx"]
backend = GenericBackendV2(num_qubits=5, coupling_map=coupling_map, basis_gates=basis_gates)

# Visualize the device connectivity
plot_gate_map(backend)

### Compile a 3-Qubit GHZ Circuit

Let's compile a simple 3-qubit GHZ state for this device:

In [ ]:
from qiskit import transpile, QuantumCircuit
from mqt.bench import get_benchmark, BenchmarkLevel

# Assemble a 3 qubit GHZ state
circ = QuantumCircuit(3)
circ.h(0)
circ.cx(0,1)
circ.cx(0,2)
circ.measure_all()

circ.draw(output='mpl')

In [ ]:
# Compile the circuit to the device using Qiskit's transpiler
circ_mapped = transpile(circ, backend=backend, optimization_level=3)

In [ ]:
# Visualize the compiled circuit
circ_mapped.draw(output='mpl')

### Compile a 5-Qubit GHZ Circuit

Now let's try a circuit that uses all qubits on the device:

In [ ]:
# Load and compile a 5-qubit GHZ circuit
circ = get_benchmark(benchmark="ghz", level=BenchmarkLevel.ALG, circuit_size=5)
circ_mapped = transpile(circ, backend=backend, optimization_level=3)
circ_mapped.draw(output='mpl')

### Compile a More Complex Circuit

Let's try Grover's algorithm, which requires more complex gate interactions:

In [ ]:
# Load a 5-qubit Grover circuit and decompose it
circ = get_benchmark(benchmark="grover", level=BenchmarkLevel.ALG, circuit_size=5).decompose(reps=3)
circ.draw(output="mpl", fold=100)

In [ ]:
# Compile to the device using Qiskit
circ_mapped = transpile(circ, backend=backend, optimization_level=3)
circ_mapped.draw(output="mpl", fold=100)

In [ ]:
# Compare depth of compiled and uncompiled circuits
circ_mapped.count_ops()
print("Uncompiled circuit depth: ", circ.depth())
print("Compiled circuit depth: ", circ_mapped.depth())

## Neutral atom compilation

Neutral-atom platforms offer flexible qubit layouts by dynamically rearranging atoms with optical tweezers. This enables effective all-to-all connectivity, reducing the need for swap-heavy routing strategies.  

Specialized entanglement zones permit multi-qubit interactions. A compiler must therefore determine when and how to transport atoms efficiently between interaction and storage regions.

### Define a Neutral Atom Architecture

Create an architecture with one storage zone and one entanglement zone:

In [ ]:
from mqt.qmap.na.zoned import ZonedNeutralAtomArchitecture

arch = ZonedNeutralAtomArchitecture.from_json_string("""{
  "name": "Architecture with one entanglement and one storage zone",
  "operation_duration": {"rydberg_gate": 0.36, "single_qubit_gate": 52, "atom_transfer": 15},
  "operation_fidelity": {"rydberg_gate": 0.995, "single_qubit_gate": 0.9997, "atom_transfer": 0.999},
  "qubit_spec": {"T": 1.5e6},
  "storage_zones": [{
    "zone_id": 0,
    "slms": [{"id": 0, "site_separation": [3, 3], "r": 20, "c": 100, "location": [0, 0]}],
    "offset": [0, 0],
    "dimension": [297, 57]
  }],
  "entanglement_zones": [{
    "zone_id": 0,
    "slms": [
      {"id": 1, "site_separation": [12, 10], "r": 7, "c": 20, "location": [35, 67]},
      {"id": 2, "site_separation": [12, 10], "r": 7, "c": 20, "location": [37, 67]}
    ],
    "offset": [35, 67],
    "dimension": [230, 60]
  }],
  "aods": [{"id": 0, "site_separation": 2, "r": 100, "c": 100}],
  "rydberg_range": [[[30, 62], [270, 132]]]
}""")

### Create an 8-Qubit GHZ Circuit

In this example, we will demonstrate how to use the zoned neutral atom compiler to generate a sequence of
target-specific instructions for a quantum circuit.
For this purpose, we employ a circuit that prepares a Greenberger-Horne-Zeilinger (GHZ) state of 8 qubits.
Compared to the usual GHZ circuit that applies the CX-gates in a chain, this circuit applies the CX-gates in a tree-like
manner.
First, it brings one qubit into maximal superposition and then applies the first CX-gate as usual.
Afterward, it applies two CX-gates in parallel controlled on the qubits already in superposition instead of applying them
serially.

In [ ]:
from qiskit import QuantumCircuit
from numpy import pi

def global_ry(theta, num_qubits):
    """:returns: a global ry gate"""
    qc = QuantumCircuit(num_qubits)
    qc.ry(theta, range(num_qubits))
    return qc.to_gate(label = f"Ry({theta:.3f})")


qc = QuantumCircuit(8)
qc.append(global_ry(-pi/4, 8), range(8))
qc.z(range(8))
qc.append(global_ry(pi/4, 8), range(8))
qc.cz(0, 4)
qc.append(global_ry(-pi/4, 8), range(8))
qc.z(4)
qc.append(global_ry(pi/4, 8), range(8))
qc.cz(0, 2)
qc.cz(4, 6)
qc.append(global_ry(-pi/4, 8), range(8))
qc.z([2, 6])
qc.append(global_ry(pi/4, 8), range(8))
qc.cz(0, 1)
qc.cz(2, 3)
qc.cz(4, 5)
qc.cz(6, 7)
qc.append(global_ry(-pi/4, 8), range(8))
qc.z([1,3,5,7])
qc.append(global_ry(pi/4, 8), range(8))

qc.draw(output="mpl")

In [ ]:
from mqt.qmap.na.zoned import RoutingAgnosticCompiler, RoutingAwareCompiler

compiler = RoutingAwareCompiler(arch)
# or if you want to use the routing-agnostic compiler:
# compiler = RoutingAgnosticCompiler(arch)

### Compile with Routing-Aware Compiler

The routing-aware compiler optimizes atom movements based on the circuit structure:

In [ ]:
from mqt.core import load

circ = load(qc)
code = compiler.compile(circ)
print(code)

The compiler can be configured with different parameters, e.g., the deepening factor of the A*-placer:

In [ ]:
compiler = RoutingAwareCompiler(arch, deepening_factor = 0.6)

circ = load(qc)
code = compiler.compile(circ)
print(code)

# save to .naviz file
with open("compiled_circuit.naviz", "w") as f:
    f.write(code)

## Visualizing with NAViz

The compilation output is in a `.naviz` string, that can be opened with NAViz.
Simply copy past the output sting into a browser window or import the saved `.naviz` file there:

https://munich-quantum-toolkit.github.io/naviz/

## Compilation for Neutral Atom Quantum Computers

Neutral atom platforms offer unique advantages through dynamic atom rearrangement:tasks:

- Choose different algorithms from MQT Bench (see https://mqt.readthedocs.io/projects/bench/en/latest/Benchmark_selection.html)
- Adjust size and connectivity of considered backend (see https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.CouplingMap -> e.g., from_line, from_ring, from_full)